In [ ]:
root_path = ""
input_len = 96
output_len = 72
input_dim = 18
enc_hidden = 18
dec_hidden = 18
num_levels = 2
dropout = 0.5
K_IMP = 6
num_heads = 1
batch_size = 32
epochs = 10
learning_rate = 0.01

# Dataset and DataLoader
train_dataset = Dataset_Custom(root_path=root_path, flag='train', size=(input_len, 24, output_len - 24),
                               features='M', data_path='Data2.csv', target='Kp', scale=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, and Optimizer
model = DPAD_ATT(input_len=input_len, output_len=output_len, input_dim=input_dim,
                 enc_hidden=enc_hidden, dec_hidden=dec_hidden, num_levels=num_levels,
                 dropout=dropout, K_IMP=K_IMP, num_heads=num_heads)
model = model.cuda()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for i, (seq_x, seq_y) in enumerate(train_loader):
        seq_x, seq_y = seq_x.float().cuda(), seq_y.float().cuda()

        optimizer.zero_grad()

        print(f"Batch {i + 1} - Input shape: {seq_x.shape}")
        print(f"Batch {i + 1} - Target shape: {seq_y.shape}")

        output = model(seq_x)

        print(f"Batch {i + 1} - Output shape: {output.shape}")

        loss = criterion(output, seq_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if i % 10 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Batch {i + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')

    print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {epoch_loss / len(train_loader):.4f}')

print('Training complete')